In [2]:
import nltk
# nltk.download()

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
from sklearn.datasets import load_files

stopwords = nltk.corpus.stopwords.words('english')


types = pd.read_csv('mbti_1.csv')
types.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [4]:
# types['posts'].iloc[0]

In [14]:
#average length of posts not counting white spaces
#calculate percent of punctuation in post column

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

types['punct%'] = types["posts"].apply(lambda x: count_punct(x))
types["body_length"] = types['posts'].apply(lambda x: (len(x) - x.count(" "))/50)

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(types[['posts', 'body_length', 'punct%']], types['type'], test_size=0.2)

In [23]:
#instantiate and then pass in clean_text to the analyzer
#fit only on training data

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['posts'])

tfidf_train = tfidf_vect_fit.transform(X_train['posts'])
tfidf_test = tfidf_vect_fit.transform(X_test['posts'])

X_train_vect = pd.concat([X_train[["body_length", "punct%"]].reset_index(drop=True),
           pd.DataFrame(tfidf_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[["body_length", "punct%"]].reset_index(drop=True),
           pd.DataFrame(tfidf_train.toarray())], axis=1)

X_train_vect.head()

MemoryError: Unable to allocate 13.6 GiB for an array with shape (6940, 262481) and data type float64

In [ ]:
from sklearn.ensembel import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time